In [0]:
from pyspark.sql.functions import datediff, current_date, avg
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, max, min, when, substring, upper

In [0]:
# 1. What was the average time each driver spent at the pit stop for each race?
df_pitstops = spark.read.csv('s3://columbia-gr5069-main/raw/pit_stops.csv', header=True)
display(df_pitstops)

In [0]:
df_avg_duration = df_pitstops.groupby('raceId', 'driverId').agg(avg('milliseconds'))
display(df_avg_duration)

In [0]:
# 2.  Rank the average time spent at the pit stop in order of who won each race
df_laptimes = spark.read.csv('s3://columbia-gr5069-main/raw/lap_times.csv', header=True)
display(df_laptimes)

In [0]:
df_driver = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True)
display(df_driver)

In [0]:
from pyspark.sql.functions import col, max

df_laptimes = df_laptimes.withColumn("lap", col("lap").cast("integer"))
max_laps = df_laptimes.groupBy("raceId", "driverId").agg(max("lap").alias("last_lap"))
display(max_laps)


In [0]:
final_positions_df = max_laps.join(
    df_laptimes,
    (max_laps.raceId == df_laptimes.raceId) & 
    (max_laps.driverId == df_laptimes.driverId) & 
    (max_laps.last_lap == df_laptimes.lap),
    "inner")
display(final_positions_df)

In [0]:
df_rank_avgpitstop = df_avg_duration.join(final_positions_df, on=['driverId', 'raceId']).orderBy(col("position").cast("int").asc())
display(df_rank_avgpitstop)

In [0]:
# 3.  Insert the missing code (e.g: ALO for Alonso) for drivers based on the 'drivers' dataset
df_driver = spark.read.csv('s3://columbia-gr5069-main/raw/drivers.csv', header=True)
display(df_driver)

In [0]:
from pyspark.sql.functions import col, when, substring, upper

df_driver = df_driver.withColumn('code', upper(substring(col("surname"), 1, 3)))
display(df_driver)

In [0]:
# 4. Who is the youngest and oldest driver for each race? Create a new column called “Age”
df_driver = df_driver.withColumn("age", datediff(current_date(),df_driver.dob)/365)
display(df_driver)

In [0]:
max_age = df_driver.select(max(col("age"))).collect()[0][0]
min_age = df_driver.select(min(col("age"))).collect()[0][0]
df_max_min = df_driver.filter((col("age") == max_age) | (col("age") == min_age))
display(df_max_min)